In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import torch
from transformers import BartTokenizer,BartForConditionalGeneration, Trainer, TrainingArguments,DataCollatorWithPadding,AdamW
from datasets import Dataset

# Important Step:
please change the filename below to the file you want to use for training (This should not include the .csv)

## Load the Data

In [2]:
# Specify the training file to take. Change the hashes, filename = '###'
filename = '20K'

# Load the CSV file
csv_file = './../3. Cleaned Data/'+filename+'.csv'
df = pd.read_csv(csv_file)

In [3]:
df.head()

,Unnamed: 0,original,word_count_original,corrected_fs,word_count_corrected_fs,word_count_diff,word_count_ratio,o_pos_tags,o_num_verbs,o_num_nouns,...,c_num_modal,num_verbs_diff,num_nouns_diff,num_adjs_diff,num_adv_diff,num_pronoun_diff,num_preposition_diff,num_conjunction_diff,num_article_diff,num_modal_diff
0,539741,"Since I have begun to live in London, I have b...",16,"Since I have begun living in London, I have be...",15,-1,0.937500,"[('Since', 'IN'), ('I', 'PRP'), ('have', 'VBP'...",7,2,...,0,-1,1,0,0,0,0,0,0,0
1,805208,"If she heard my description of her, she would ...",12,"If she had heard my description of her, she wo...",13,1,1.083333,"[('If', 'IN'), ('she', 'PRP'), ('heard', 'VBD'...",2,2,...,1,1,0,0,0,0,0,0,0,0
2,553823,I never can do.,4,I could never do it.,5,1,1.250000,"[('I', 'PRP'), ('never', 'RB'), ('can', 'MD'),...",1,0,...,1,0,0,0,0,1,0,0,0,0
3,792625,In the class I learned how to write paragraph ...,10,In the class I learned how to write paragraphs.,9,-1,0.900000,"[('In', 'IN'), ('the', 'DT'), ('class', 'NN'),...",2,2,...,0,0,0,-1,0,0,0,0,0,0
4,686799,I think imitating is important thing to learn ...,9,I think imitating is important to learning Eng...,8,-1,0.888889,"[('I', 'PRP'), ('think', 'VBP'), ('imitating',...",4,2,...,0,0,-1,0,0,0,0,0,0,0


In [4]:
# Change the column names in the dataframe
df.rename(columns = {'corrected_fs':'corrected'}, inplace = True)
df=df[['original','corrected']]

In [5]:
# Split the dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

## Tokenization

In [6]:
# Instantiate the tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

In [7]:
# Create tokenization and encoding for training and test sets
train_encodings = tokenizer(list(train_df['original']), truncation=True, padding=True,return_tensors='pt')
val_encodings = tokenizer(list(val_df['original']), truncation=True, padding=True,return_tensors='pt')

train_labels = tokenizer(list(train_df['corrected']), truncation=True, padding=True,return_tensors='pt')
val_labels = tokenizer(list(val_df['corrected']), truncation=True, padding=True,return_tensors='pt')

In [8]:
tokenizer.decode(train_encodings['input_ids'][1])

'<s>Whoever I talk with, they always talking about how to lose their weight.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [9]:
tokenizer.decode(train_labels['input_ids'][1])

'<s>Whoever I talk with, they are always talking about how to lose weight.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [10]:
# Assign cuda to the device to use for training
if torch.cuda.is_available(): 
 dev = "cuda:0" 
 print("This model will run on CUDA")
elif  torch.backends.mps.is_available(): 
 dev = "mps:0"
 print("This model will run on MPS")
else:
 dev = "cpu" 
 print("This model will run on CPU")
device = torch.device(dev) 

print(device)

This model will run on CUDA
cuda:0


## Fine-tune the BART Model

In [11]:
#Create a PyTorch dataset
class TextDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels['input_ids'][idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

In [12]:
# Instantiate the model
checkpoint = "facebook/bart-large"
model = BartForConditionalGeneration.from_pretrained(checkpoint).to(device)
#model.to(device)

In [13]:
# Instantiate the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5,no_deprecation_warning=True)

In [14]:
# Train the BART model

from torch.utils.data import DataLoader
from tqdm.auto import tqdm

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch, use_cache=False)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        progress_bar.update(1)
    print("Epoch {} train loss: {}".format(epoch, train_loss / len(train_loader)))

  0%|          | 0/1689 [00:00<?, ?it/s]

/tmp/ipykernel_2019/551999556.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_2019/551999556.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])


Epoch 0 train loss: 2.758947445857165
Epoch 1 train loss: 0.2003404620644466
Epoch 2 train loss: 0.16016714388693204


In [26]:
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

model.eval()
val_loss = 0
reference_corpus = []
predicted_corpus = []

num_validation_steps = len(val_loader)
progress_bar = tqdm(range(num_validation_steps))

for batch in val_loader:
    batch = {k: v.to(device) for k, v in batch.items()}

    # Store the labels in a separate variable and remove labels from the batch
    labels = batch['labels']
    batch.pop('labels')

    with torch.no_grad():
        outputs = model.generate(**batch, use_cache=False)
        for i in range(len(outputs)):
            predicted_sentence = tokenizer.decode(outputs[i], skip_special_tokens=True)
            reference_sentence = tokenizer.decode(labels[i], skip_special_tokens=True)
            reference_corpus.append([reference_sentence.split()])
            predicted_corpus.append(predicted_sentence.split())
            val_loss += model(**batch, use_cache=False, labels=labels).loss.item()
            progress_bar.update

/tmp/ipykernel_2019/551999556.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_2019/551999556.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])
/home/timlinux/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [27]:
reference_corpus

[[['The',
   'title',
   'of',
   'the',
   'song',
   'is',
   'and',
   'it',
   'was',
   'released',
   'about',
   '20years',
   'ago.']],
 [['So', 'I', "can't", 'correct', 'many', 'entries', 'today.']],
 [['So', "I'm", 'looking', 'forward', 'to', 'going', 'there.']],
 [['It', 'is', 'flavored', 'with', 'sweet', 'soy', 'sauce.']],
 [['Does', 'it', 'mean', 'my', 'love', 'will', 'go', 'well?']],
 [['Because',
   'there',
   'are',
   'too',
   'many',
   'people',
   'for',
   'it',
   'to',
   'feel',
   'natural.']],
 [['I',
   'wonder',
   'if',
   'I',
   'profit,',
   'how',
   'well',
   'will',
   'the',
   'charity',
   'do?']],
 [["I'm", 'a', 'little', 'tired', 'this', 'week.']],
 [['Usually,',
   'I',
   "don't",
   'have',
   'any',
   'interest',
   'in',
   'this',
   'kind',
   'of',
   'group',
   'or',
   'music,',
   'but',
   'one',
   'day',
   'I',
   'happened',
   'to',
   'watch',
   'a',
   'TV',
   'show',
   'focusing',
   'on',
   'it',
   'and',
   'was',


In [28]:
predicted_corpus

[['The',
  "song's",
  'title',
  'is',
  'and',
  'it',
  'was',
  'released',
  'about',
  '20',
  'years',
  'ago.'],
 ['However', 'I', "can't", 'correct', 'many', 'entries', 'today.'],
 ['Therefore', "I'm", 'looking', 'forward', 'to', 'going', 'there.'],
 ['It', 'is', 'flavored', 'with', 'sweet', 'soy', 'sauce.'],
 ['it', 'means', 'my', 'love', 'will', 'go', 'well?'],
 ['Because', 'it', 'is', 'too', 'many', 'people', 'to', 'feel', 'nature.'],
 ['I',
  'wonder',
  'if',
  'I',
  'gain',
  'a',
  'profit,',
  'does',
  'it',
  'make',
  'good',
  'as',
  'a',
  'charity?'],
 ["I'm", 'a', 'little', 'tired', 'this', 'week.'],
 ['Usually,',
  'I',
  "don't",
  'have',
  'any',
  'intrest',
  'in',
  'this',
  'kind',
  'of',
  'group',
  'or',
  'music,'],
 ['My', 'dogs', 'seem', 'to', 'feel', 'jealousy,', 'though.'],
 ['Tonight',
  'is',
  'a',
  'slow',
  'night',
  'and',
  'good',
  'night',
  'my',
  'friends'],
 ['The', 'plan', 'of', 'the', 'business', 'trip', 'abroad', 'was', 'de

In [ ]:
# Save the trained model and tokenizer
output_dir = "../7. Models/"+filename+"_"+checkpoint"/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
